In [1]:
# UNCOMMENT THESE TO INSTALL THE REQUIRED LIBRARIES

# %pip install pandas
# %pip install tensorflow
# %pip install scikit-learn
# %pip install matplotlib

In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from matplotlib import pyplot as plt 

import warnings
warnings.filterwarnings("ignore")

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
DATAPATH = "../input/megaGymDataset.csv"

# Load the dataset
dataset = pd.read_csv(DATAPATH)
dataset.fillna('', inplace=True)
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,4.5
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,4.5
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,4.5
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.5


In [4]:
def recommend_exercises(user_input):
    data = pd.read_csv(DATAPATH)
    
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])
    
    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Build the ANN model using TensorFlow
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(data["ID"].unique()), activation='softmax')  # Use the unique number of IDs as the output dimension
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    
    user_input_encoded = [label_encoders[feature].transform([user_input[i]])[0] for i, feature in enumerate(["Type", "BodyPart", "Equipment", "Level"])]
    user_input_scaled = scaler.transform([user_input_encoded])
    
    predictions = model.predict(user_input_scaled)
    
    return data.iloc[np.squeeze(predictions.argsort())][::-1][:10].sort_values(["Rating"], ascending=False).index.to_list()

In [5]:
user_input = ["Strength", "Biceps", "Body Only", "Intermediate"]
IDs = recommend_exercises(user_input)
dataset.iloc[IDs]

Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: 7.9894 - val_accuracy: 0.0000e+00 - val_loss: 8.0160
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0058 - loss: 7.8970 - val_accuracy: 0.0000e+00 - val_loss: 8.7031
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0041 - loss: 6.9442 - val_accuracy: 0.0000e+00 - val_loss: 10.2760
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0096 - loss: 6.1666 - val_accuracy: 0.0000e+00 - val_loss: 12.6001
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0160 - loss: 5.6634 - val_accuracy: 0.0000e+00 - val_loss: 14.0917
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0252 - loss: 5.3538 - val_accuracy: 0.0000e+00 - val_loss: 15.7686
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0269 - loss: 5.0584 - val_accuracy: 0.0000e+00 - val_loss: 17.0900
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0379 - loss: 4.88

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
854,854,Pull-up - Gethin Variation,The pull-up is a multi-joint bodyweight exerci...,Strength,Biceps,Body Only,Intermediate,9.000000
857,857,UN Pull-Up Neutral,"Pull-up variation with a neutral grip, targeti...",Strength,Biceps,Body Only,Intermediate,9.000000
855,855,UN Pull-Up Supinated,Pull-up variation with a supinated (underhand)...,Strength,Biceps,Body Only,Intermediate,9.000000
853,853,TBS Chin-Up,The a chin-up is a variation of the pull-up ex...,Strength,Biceps,Body Only,Intermediate,9.000000
910,910,Standing calf raise-,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833
911,911,HM Left Calf-Spike Sumo Pulse,Dynamic calf exercise involving pulsing moveme...,Strength,Calves,Body Only,Intermediate,6.870833
909,909,Standing Calf Raise,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833
697,697,Side-lying leg adduction scissor,The side-lying leg adduction scissor is a dyna...,Strength,Adductors,Body Only,Intermediate,4.816667
695,695,UNS Foam Roll Adductor,Using a foam roller to massage and release ten...,Strength,Adductors,Body Only,Intermediate,4.816667
684,684,Side-To-Side Adductor Stretch,Stretching the adductor muscles by moving legs...,Strength,Adductors,Body Only,Intermediate,4.816667


In [6]:
IDs

[854, 857, 855, 853, 910, 911, 909, 697, 695, 684]